In [6]:
!pip install -r ../requirements.txt
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import requests
import json
from datetime import date, datetime
import boto3
from io import StringIO
from dotenv import load_dotenv
import os
import time

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached pandas-1.5.0-cp310-cp310-win_amd64.whl (10.4 MB)
  Using cached boto3-1.24.80-py3-none-any.whl (132 kB)
  Using cached selenium-4.4.3-py3-none-any.whl (985 kB)
  Using cached webdriver_manager-3.8.3-py2.py3-none-any.whl (26 kB)
  Using cached pytz-2022.2.1-py2.py3-none-any.whl (500 kB)
  Using cached numpy-1.23.3-cp310-cp310-win_amd64.whl (14.6 MB)
  Using cached s3transfer-0.6.0-py3-none-any.whl (79 kB)
     ---------------------------------------- 9.2/9.2 MB 4.9 MB/s eta 0:00:00
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached trio-0.21.0-py3-none-any.whl (358 kB)
     -------------------------------------- 161.1/161.1 kB 4.9 MB/s eta 0:00:00
     -------------------------------------- 140.4/140.4 kB 4.2 MB/s eta 0:00:00
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     ---------------------------------------- 62.8/62

In [7]:
load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")

In [ ]:
s3_client = boto3.client('s3')

response = s3_client.get_object(Bucket = AWS_S3_BUCKET, Key = 'raw/tracker_player_rayzem_2022-09-26 17:54:44.873258.json')

# data = response['Body'].read()

data = response['Body'].read()


data

In [158]:
df_profile = pd.read_json(StringIO(data.decode('utf-8')))
# df_profile = pd.read_json(StringIO(data))

# df_profile = pd.json_normalize(json.loads(data))
# df_profile['data.matches']
# df_profile = df_profile[['data.matches']]
# df_profile_1 = json.loads(df_profile.to_json(orient='records'))
# df_profile_2= json.loads(json.dumps(df_profile_1))
# df_profile_3 = pd.read_json(df_profile_2)

df_profile


,data
expiryDate,2022-09-26T20:57:38.7444429+00:00
matches,[{'attributes': {'id': 'f68f3277-b067-4788-b36...
metadata,"{'schema': 'riot-api', 'next': 1}"
paginationType,Next
requestingPlayerAttributes,"{'platformSlug': 'riot', 'platformUserIdentifi..."


In [39]:
print(df_profile.columns)

Index(['data', 'data.schema', 'data.next', 'data.platformSlug',
       'data.platformUserIdentifier', 'data.attributes.id',
       'data.attributes.mapId', 'data.attributes.modeId',
       'data.metadata.modeKey', 'data.metadata.modeName',
       'data.metadata.modeImageUrl', 'data.metadata.modeMaxRounds',
       'data.metadata.isAvailable', 'data.metadata.timestamp',
       'data.metadata.result', 'data.metadata.map', 'data.metadata.mapName',
       'data.metadata.mapImageUrl', 'data.segments', 'data.expiryDate'],
      dtype='object')


In [45]:
df_profile = df_profile['data.segments']

df_profile

0                                                   NaN
1     [{'type': 'overview', 'attributes': {'platform...
2     [{'type': 'overview', 'attributes': {'platform...
3     [{'type': 'overview', 'attributes': {'platform...
4     [{'type': 'overview', 'attributes': {'platform...
5     [{'type': 'overview', 'attributes': {'platform...
6     [{'type': 'overview', 'attributes': {'platform...
7     [{'type': 'overview', 'attributes': {'platform...
8     [{'type': 'overview', 'attributes': {'platform...
9     [{'type': 'overview', 'attributes': {'platform...
10    [{'type': 'overview', 'attributes': {'platform...
11    [{'type': 'overview', 'attributes': {'platform...
12    [{'type': 'overview', 'attributes': {'platform...
13    [{'type': 'overview', 'attributes': {'platform...
14    [{'type': 'overview', 'attributes': {'platform...
15    [{'type': 'overview', 'attributes': {'platform...
16    [{'type': 'overview', 'attributes': {'platform...
17    [{'type': 'overview', 'attributes': {'plat

In [48]:
df_profile = pd.json_normalize(json.loads(df_profile.to_json()))

df_profile

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,None,"[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...",...,"[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...","[{'type': 'overview', 'attributes': {'platform...",None,None,None
